In [1]:
# Import necessary modules
import pandas as pd

# This function reads the file and ignores the columns
# that has most of the empty values.


def trim_empty(file):
    df = pd.read_csv(file, skiprows=1, low_memory=False)
    cols = df.count()
    cols_delete = cols <= cols.max()/3
    cols_delete = cols[cols_delete]
    df.drop(list(cols_delete.index), axis=1, inplace=True)
    return df


def trim_empty_df(df):
    cols = df.count()
    cols_delete = cols <= cols.max()/3
    cols_delete = cols[cols_delete]
    df.drop(list(cols_delete.index), axis=1, inplace=True)
    return df

In [2]:
# Load datas
df2018_3 = trim_empty('Original_data\LoanStats_2018Q3.csv')
df2018_2 = trim_empty('Original_data\LoanStats_2018Q2.csv')
df2018_1 = trim_empty('Original_data\LoanStats_2018Q1.csv')

df2017_4 = trim_empty('Original_data\LoanStats_2017Q4.csv')
df2017_3 = trim_empty('Original_data\LoanStats_2017Q3.csv')
df2017_2 = trim_empty('Original_data\LoanStats_2017Q2.csv')
df2017_1 = trim_empty('Original_data\LoanStats_2017Q1.csv')

df2016_4 = trim_empty('Original_data\LoanStats_2016Q4.csv')
df2016_3 = trim_empty('Original_data\LoanStats_2016Q3.csv')
df2016_2 = trim_empty('Original_data\LoanStats_2016Q2.csv')
df2016_1 = trim_empty('Original_data\LoanStats_2016Q1.csv')

df_d = trim_empty('Original_data\LoanStats3d.csv')
df_c = trim_empty('Original_data\LoanStats3c.csv')
df_b = trim_empty('Original_data\LoanStats3b.csv')
df_a = trim_empty('Original_data\LoanStats3a.csv')

# Combine different data into a single dataframe.
# Again, we ignore the columns that is mostly empty.
df = pd.concat([df2018_3, df2018_2, df2018_1,
                df2017_4, df2017_3, df2017_2, df2017_1,
                df2016_4, df2016_3, df2016_2, df2016_1,
                df_a, df_b, df_c, df_d], axis=0, ignore_index=True, sort=False)

df = trim_empty_df(df)

In [3]:
# Convert colums with data into python datetime object
df.earliest_cr_line = pd.to_datetime(df.earliest_cr_line)
df.issue_d = pd.to_datetime(df.issue_d)
df.last_pymnt_d = pd.to_datetime(df.last_pymnt_d)
df.last_credit_pull_d = pd.to_datetime(df.last_credit_pull_d)

In [4]:
# Some columns contain characters such as % and symbols.
# We need to delete them.

df.int_rate = pd.to_numeric(df.int_rate.str.strip('%'))

df.term = pd.to_numeric(df.term.str.strip('months'))

df1 = df.emp_length.str.replace('+', '')
df1 = df1.str.replace('<', '')
df.emp_length = pd.to_numeric(df1.str.strip('years'))

df.revol_util = df.revol_util.str.strip('%').astype('float')

In [5]:
# Loan status column has multiple categories.
# We simplyfy those into two categories.
condition1 = pd.Series(df['loan_status']
                       .str.contains('Late')).astype('bool')
condition2 = pd.Series(df['loan_status']
                       .str.contains('Charged')).astype('bool')
condition = condition1 | condition2

df.loan_status = df.loan_status.where(~condition, 'Late/Charged Off')

condition1 = pd.Series(df['loan_status']
                       .str.contains('Current')).astype('bool')
condition2 = pd.Series(df['loan_status']
                       .str.contains('Paid')).astype('bool')
condition3 = pd.Series(df['loan_status']
                       .str.contains('Grace')).astype('bool')
condition4 = pd.Series(df['loan_status']
                       .str.contains('Default')).astype('bool')

condition = condition1 | condition2 | condition3 | condition4

df.loan_status = df.loan_status.where(~condition, 'Current/Fully Paid')

condition = df.annual_inc > 5*df.annual_inc.mean()
df.annual_inc = df.annual_inc.where(~condition, df.annual_inc.mean())

df_bool = df.verification_status == 'Source Verified'
df.verification_status = df.verification_status.where(~df_bool, 'Verified')

condition = (df.home_ownership == 'OTHER') |\
            (df.home_ownership == 'ANY') |\
            (df.home_ownership == 'NONE')
        
df.home_ownership = df.home_ownership.where(~condition, None)

In [6]:
# Delete unnecessary columns
df.drop(['grade', 'sub_grade', 'zip_code', 'policy_code', 'title'],
        axis=1, inplace=True)

In [7]:
 # Save data to csv for future analysis. 
df = df[df.funded_amnt_inv >= 1000]

df.to_csv('Intermediate_data/Trimmed.csv')